# Clustering examples

This notebook is for demonstrating how to use the fuzzy water clustering package with scikit-learn.

The basic idea is that we create some scikit-learn compatible clustering estimators and a group of scoring functions. They can then be thrown about using scikit-learn for the following important purposes.

* [Pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) : Chain together pre-processing, clustering and scoring steps into one object
* [Model evaluation](https://scikit-learn.org/stable/model_selection.html#model-selection) : Cross validation of models to find optimum fitting parameters as scored against various scoring functions

We aim to build clustering estimators, store them in ./Models. Additional capability we are looking to add:

* Ensemble methods (run many times to assess stability)
* I/O helper functions (read csv, netcdf, whatever)
* Suite of scoring metrics (fuzzy silouette score, partition coefficient, etc)
* Default, pretrained models (one for each common dataset, like OLCI, OC-CCI etc)

in a way that is generalised to work on all models. Must be a scikit-learn compatible object and must not (!) reinvent wheels here

Note, best practice is to install dependencies concurrently in case they have not loaded correctly from pip install of package:
* conda install notebook numpy xarray dask scikit-learn scikit-fuzzy hvplot holoviews datashader netcdf4 h5netcdf

In [ ]:
# import the whole package
import fuzzy_water_clustering as fwc

In [ ]:
# for serialization of arbitrary Python objects
import pickle
import joblib

# scikit-learn objects for many things
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score #, pairwise_distances, make_scorer, 
# from sklearn.metrics.cluster import calinski_harabasz_score, davies_bouldin_score, silhouette_samples, silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

# for data handling and visualization
import pandas as pd
import hvplot.pandas
import xarray as xr
import hvplot.xarray
import holoviews as hv
import numpy as np
from functools import reduce

Create a dummy data set

In [ ]:
blobs, labels = make_blobs(n_samples=2000, n_features=11)
dfb = pd.DataFrame(blobs.T)

In [ ]:
dfb.head(10)

# 1 using cluster estimator

In [ ]:
# create a instance of cmeans model, with number of clusters c
cmeans = fwc.CmeansModel(n_clusters=3)

In [ ]:
# fit against the dataset, this generates a cluster set
cmeans.fit(blobs)

In [ ]:
df_cntr = pd.DataFrame(cmeans.cluster_centers_.T)

In [ ]:
dfb.T[labels == 0].T.hvplot(kind='line',datashade=True, cmap='Reds', label='Cluster 1') *\
dfb.T[labels == 1].T.hvplot(kind='line',datashade=True, cmap='Blues', label='Cluster 2') *\
dfb.T[labels == 2].T.hvplot(kind='line',datashade=True, cmap='Greens', label='Cluster 3')

In [ ]:
reduce(lambda a,b:a*b, [dfb.T[labels == i].mean().T.hvplot(c=c) for (i,c) in zip(range(3),['red','blue','green'])]).opts(title='cluster centres')

In [ ]:
# plot the classified data
df_cntr.hvplot(title='cluster centres found')

# 2 Performance metrics

if an algorithm comes with its own scoring metric that will be class specific. However, we can get some metrics that are applicable across all methods. 

Scikit-learn has a [suite of metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)

Probably the best way is to make a scorer object from a scoring function with `sklearn.metrics.make_scorer` which can be placed at after clustering in a pipeline which is then fed to a GridSearch like object.

In [ ]:
ss = fwc.cluster_scoring.silhouette_samples(blobs, cmeans.labels_)
hv.Bars(
    np.hstack([sorted(ss[cmeans.labels_==i]) for i in range(3)]),
).opts(title=f"silhouette_score = {np.round(silhouette_score(blobs, cmeans.labels_), 10)}")

# 3 Pipelines: chaining transforms and estimators together

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

https://scikit-learn.org/stable/modules/compose.html

In [ ]:
# easy to make a pipeline with whatever pieces you want
pl = make_pipeline(
    StandardScaler(),
    PCA(),
    fwc.CmeansModel(n_clusters=3)     
)

In [ ]:
pl.fit(blobs)

### 3.x inverse_transforms to get final results in original space

Each transform step in a pipeline has an inverse_transform method. Enabling us to reverse the trasform so that final results can be projected into the original space.

In this example, scaling of the data and principal component analysis are applied before clustering. Therefore the clusters are defined in a transformed space. Using inverse_transforms we regain the orginal x,y space the data came in.

In [ ]:
df_cntr = pd.DataFrame(
    pl['standardscaler'].inverse_transform(
        pl['pca'].inverse_transform(
            pl['cmeansmodel'].cluster_centers_
        )
    )
)

In [ ]:
dfb.hvplot(kind='line', datashade=True, width=1000) * \
df_cntr.T.hvplot(kind='line')

# 4 GridSearchCV: Exhaustive search to find best fitting parameters

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

One problem we have is that the clustering algorithms performance depends on the fitting parameters (hyper paramters). These are not learnt, rather they determine the behaviour of the algorithm when finding the learnt parameters. GridSearchCV helps by exhaustively fitting the model to the data for every combination of fitting paramters supplied to it. Furthermore, it fits and scores the model to 5 subsets of the data (by default) so that the variance of the score can be measured.

In [ ]:
# define a dict of parameters to go through all combinations of values.
# double underscore joins the step name to its parameter
param_grid = {
    'cmeansmodel__c': [2,4,6,8,10],
    'cmeansmodel__m':[1.5,2.0,2.5],
}

In [ ]:
# define the scoring metrics you want to evaluate
scoring = {
    'XB': fwc.cluster_scoring.xie_beni,
    'SIL': fwc.cluster_scoring.hard_silouette,
    'FPC': fwc.cluster_scoring.fuzzy_partition_coef,
    'DB': fwc.cluster_scoring.davies_bouldin,
}

In [ ]:
# create a grid search cross validation object
gs = GridSearchCV(
    pl,
    param_grid=param_grid,
    scoring=scoring,
    refit='XB',
    n_jobs=4
)

In [ ]:
# fitting syntax is just like before. Except now it tries all combinations of parameters,
# scores and refits witht he best according to the chosen refit metric.
gs.fit(blobs)

In [ ]:
# create a pandas.DataFrame that contains the results of the fitting
dfgs = pd.DataFrame(gs.cv_results_)

## 5.x plotting 

In [ ]:
reduce(lambda a,b:a*b,[dfgs.hvplot(
        col='param_cmeansmodel__m',
        row=None,
        kind='scatter',
        x='param_cmeansmodel__c',
        y=f'mean_test_{x}',
        label=f'{x}',
        legend=False,
        ylim=[-10,2.5],
        ylabel="score",
        xlabel=r"# clusters",
) for x in scoring.keys()]).opts(title="3 blobs, 2000 points", show_legend=True)

# 5 serializing models

If we like our results and we want to save them. There exist a few choices;

* pickle / joblib : stores arbitrary python objects in instruction orientated file. Insecure on loading, not cross platform nor cross version supported. Only open trusted files and best used short term.
* custom serialization : pipeline parameters can be stored to netcdf using utils/serialize_models. But it also isn't cross platform nor cross version supported at present.

## 5.1 pickle/joblib

In [ ]:
# store a model as a pickle file
pickle.dump(
    pl,
    open(
        "practice_fitted_pipeline.p",
        "wb"
    )
)

# 6 Application to OLCI data

In [ ]:
#THREDDS pathway for Plymouth Sound
GEO_DAILY1km_THREDDS_string = 'http://rsg.pml.ac.uk/thredds/dodsC/SENTINEL3A_OLCI-G13_300m_02-1d'
# GEO_DAILY1km_THREDDS_string = '../../data/OLCI_RRS_spectral_library_968349_pixels_from_6000_random_G25_300m_files.nc'
ds = xr.open_dataset(GEO_DAILY1km_THREDDS_string)

In [ ]:
ds

In [ ]:
ds = ds.isel(time=0)

In [ ]:
df = ds.to_dataframe()

In [ ]:
df = df.reset_index().pivot_table(index='pixel',columns='wavelength',values='Rrs')

In [ ]:
pl.fit(df)

In [ ]:
df_cntr = pd.DataFrame(
    pl['standardscaler'].inverse_transform(
        pl['pca'].inverse_transform(
            pl['cmeansmodel'].cntr_
        )
    ),
)

In [ ]:

dfb.hvplot(kind='scatter',x='x',y='y', alpha=0.5, s=1) * \
df_cntr.reset_index().hvplot(kind='scatter',x='x',y='y',c='index', cmap='rainbow', s=30)